In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
import sys, os, pathlib

import numpy as np
import pandas as pd
import torch
import warp as wp
wp.config.quiet = True

sys.path.append('../..')
import project
from project.core.utils import pprint

torch.cuda.is_available()

True

In [4]:
config = project.core.fileio.load_json('2025-11-05_config.json')
config

Loading 2025-11-05_config.json


{'dataset': 'shapenet',
 'data_root': '/ocean/projects/asc170022p/mtragoza/lung-project/data/ShapeNetSem',
 'examples': {'unit_policy': 'prefer_metadata',
  'default_unit': 0.01,
  'volume_tag': 'volume_vr',
  'mat_fields_tag': 'mat_fields_vr',
  'sim_fields_tag': 'sim_fields_vr',
  'img_fields_tag': 'img_fields_vr',
  'material_tag': 'material_vr',
  'image_tag': 'solid_texture_vr'},
 'preprocess': {'preprocess_binary_mask': {'pad_factor': 0.37},
  'preprocess_surface_mesh': {},
  'create_mesh_region_mask': {},
  'create_volume_mesh_from_mask': {'use_affine_spacing': False,
   'pygalmesh_kws': {'max_facet_distance': 0.75,
    'max_cell_circumradius': 5.0,
    'lloyd': True,
    'odt': True}},
  'create_material_mask': {'sample_kws': {'vote_rate': 0.0001}},
  'create_mesh_fields': {},
  'simulate_displacement_field': {'nu_value': 0.4,
   'scalar_degree': 0,
   'vector_degree': 1,
   'solver_kws': {}},
  'generate_volumetric_image': {'annot_path': '/ocean/projects/asc170022p/mtragoza/lu

In [5]:
csv_path = '2025-11-05_optimized.csv'
sel = pd.read_csv(csv_path)
sel

,dataset,subject,variant,method,u.norm,u.rms,u.rmse,u.nrmse,u.pcorr,u.scorr,...,u_isna,E_isna,corr_isna,res_isna,ucorr_isna,Ecorr_isna,res_order,u_order,E_order,failure_type
0,shapenet,wss.100f39dce7690f59efb94709f30ce0d2,2025-11-02,optimize,0.670816,1.128973,0.001666,0.001476,0.999999,0.997333,...,False,False,False,False,False,False,-0.0,-3.0,-1.0,ok
1,shapenet,wss.101354f9d8dede686f7b08d9de913afe,2025-11-02,optimize,0.622003,0.974864,0.001750,0.001795,0.999998,0.998717,...,False,False,False,False,False,False,-1.0,-3.0,-1.0,ok
2,shapenet,wss.1029fd5d71a06ea32ea98d69e91ba870,2025-11-02,optimize,0.849837,1.404901,0.002643,0.001881,0.999998,0.996461,...,False,False,False,False,False,False,0.0,-3.0,1.0,ok
3,shapenet,wss.102a6b7809f4e51813842bc8ef6fe18,2025-11-02,optimize,0.084750,0.128439,0.000404,0.003148,0.999994,0.997781,...,False,False,False,False,False,False,-1.0,-3.0,1.0,ok
4,shapenet,wss.107637b6bdf8129d4904d89e9169817b,2025-11-02,optimize,0.077659,0.257490,0.000958,0.003721,0.999993,0.973606,...,False,False,False,False,False,False,0.0,-2.0,-0.0,ok
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,shapenet,wss.d81f943065391492a7e760c0b399301,2025-11-02,optimize,0.054318,0.143144,0.000460,0.003214,0.999995,0.991761,...,False,False,False,False,False,False,-2.0,-2.0,-0.0,ok
1111,shapenet,wss.d8d4004791c4f61b80fa98b5eeb7036c,2025-11-02,optimize,113.014772,188.533491,0.704888,0.003740,0.999992,0.995279,...,False,False,False,False,False,False,-2.0,-2.0,-1.0,ok
1112,shapenet,wss.e8add7acb28e2505fd40810f48721e8b,2025-11-02,optimize,0.205160,0.455321,0.000704,0.001546,0.999999,0.999229,...,False,False,False,False,False,False,0.0,-3.0,-0.0,ok
1113,shapenet,wss.f6420a3a3addd027f67be63aca8832dd,2025-11-02,optimize,0.108818,0.230946,0.000418,0.001809,0.999998,0.993666,...,False,False,False,False,False,False,-0.0,-3.0,-0.0,ok


In [11]:
%autoreload
ds = project.datasets.shapenet.ShapeNetDataset(data_root='../../data/ShapeNetSem')
ds.load_metadata()

subjects = list(sel.subject)
subjects = ['wss.1e5abf0465d97d826118a17db9de8c0']

examples = list(ds.examples(subjects, variant='2025-11-02', **config['examples']))

pprint(examples[0], 2)

Example()
├── dataset:  'ShapeNet'
├── subject:  'wss.1e5abf0465d97d826118a17db9de8c0'
├── variant:  '2025-11-02'
├── paths:    dict(len=14)
|   ├── 'source_mesh':   PosixPath('../../data/ShapeNetSem/models-OBJ/models/1e5abf0465d97d826118a17db9de8c0.obj')
|   ├── 'source_mask':   PosixPath('../../data/ShapeNetSem/models-binvox-solid/1e5abf0465d97d826118a17db9de8c0.binvox')
|   ├── 'surface_mesh':  PosixPath('../../data/ShapeNetSem/2025-11-02/1e5abf0465d97d826118a17db9de8c0/meshes/surface.xdmf')
|   ├── 'binary_mask':   PosixPath('../../data/ShapeNetSem/2025-11-02/1e5abf0465d97d826118a17db9de8c0/masks/binary.nii.gz')
|   ├── 'region_mask':   PosixPath('../../data/ShapeNetSem/2025-11-02/1e5abf0465d97d826118a17db9de8c0/masks/regions.nii.gz')
|   ├── 'volume_mesh':   PosixPath('../../data/ShapeNetSem/2025-11-02/1e5abf0465d97d826118a17db9de8c0/meshes/volume_vr.xdmf')
|   ├── 'material_mask': PosixPath('../../data/ShapeNetSem/2025-11-02/1e5abf0465d97d826118a17db9de8c0/masks/material_vr.nii.g

In [12]:
project.core.fileio.load_meshio(examples[0].paths['img_fields'])

Loading ../../data/ShapeNetSem/2025-11-02/1e5abf0465d97d826118a17db9de8c0/meshes/img_fields_vr.xdmf


<meshio mesh object>
  Number of points: 3246
  Number of cells:
    tetra: 15150
  Point data: rho, E, u, bc, res, image
  Cell data: medit:ref, region, material, rho, E, image

In [13]:
from collections import defaultdict

def split_by_category(examples, test_ratio, val_ratio, seed=0):

    cats_by_subj = defaultdict(set)
    subjs_by_cat = defaultdict(set)
    for ex in examples:
        subj = ex.subject
        for cat in ex.metadata['category']:
            if cat.startswith('_'):
                continue
            cats_by_subj[subj].add(cat)
            subjs_by_cat[cat].add(subj)

    subjects = list(cats_by_subj.keys())
    n_subjects = len(subjects)
    target_test = int(round(test_ratio * n_subjects))

    categories = list(subjs_by_cat.keys())
    rng = np.random.default_rng(seed)
    rng.shuffle(categories)

    test_cats = set()
    test_subj = set()

    for cat in categories:
        proposal = test_cats | {cat}
        eligible = {s for s, c in cats_by_subj.items() if c.issubset(proposal)}
        delta = eligible - test_subj
        if len(test_subj) < target_test:
            test_cats.add(cat)
            test_subj |= delta
        else:
            break

    print( len(test_subj) / len(subjects) )
    train_subj = set(subjects) - test_subj

    num_val = int(round(val_ratio * n_subjects))
    train_list = list(train_subj)
    rng.shuffle(train_list)
    val_subj = set(train_list[:num_val])
    train_subj = set(train_list[num_val:])

    train_ex = [ex for ex in examples if ex.subject in train_subj]
    test_ex  = [ex for ex in examples if ex.subject in test_subj]
    val_ex   = [ex for ex in examples if ex.subject in val_subj]

    return train_ex, test_ex, val_ex

train_ex, test_ex, val_ex = split_by_category(examples, 0.20, 0.10, seed=0)
len(train_ex), len(test_ex), len(val_ex)

0.0


(1, 0, 0)

In [22]:
import project.datasets.torch

loader_kws = dict(
    batch_size=1,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    collate_fn=project.datasets.torch.collate_fn
)

train_set = project.datasets.torch.TorchDataset(train_ex)
train_loader = torch.utils.data.DataLoader(train_set, **loader_kws)

if test_ex:
    test_set = project.datasets.torch.TorchDataset(test_ex)
    test_loader = torch.utils.data.DataLoader(test_set, **loader_kws)
else:
    test_set = test_loader = None

if val_ex:
    val_set = project.datasets.torch.TorchDataset(val_ex)
    val_loader = torch.utils.data.DataLoader(val_set, **loader_kws)
else:
    val_set = val_loader = None


In [17]:
import project.model

model = project.model.UNet3D(
    in_channels=1,
    out_channels=1,
    n_enc_levels=2,
    n_conv_units=2,
    conv_channels=8,
    output_func='pow10'
).cuda()

n_params = project.model.count_params(model)
n_params

encoder.level0.conv_block.unit0.conv.weight        216	(8, 1, 3, 3, 3)
encoder.level0.conv_block.unit0.norm.weight        8	(8,)
encoder.level0.conv_block.unit0.norm.bias          8	(8,)
encoder.level0.conv_block.unit1.conv.weight        1728	(8, 8, 3, 3, 3)
encoder.level0.conv_block.unit1.norm.weight        8	(8,)
encoder.level0.conv_block.unit1.norm.bias          8	(8,)
encoder.level1.conv_block.unit0.conv.weight        3456	(16, 8, 3, 3, 3)
encoder.level1.conv_block.unit0.norm.weight        16	(16,)
encoder.level1.conv_block.unit0.norm.bias          16	(16,)
encoder.level1.conv_block.unit1.conv.weight        6912	(16, 16, 3, 3, 3)
encoder.level1.conv_block.unit1.norm.weight        16	(16,)
encoder.level1.conv_block.unit1.norm.bias          16	(16,)
decoder.level0.conv_block.unit0.conv.weight        5184	(8, 24, 3, 3, 3)
decoder.level0.conv_block.unit0.norm.weight        8	(8,)
decoder.level0.conv_block.unit0.norm.bias          8	(8,)
decoder.level0.conv_block.unit1.conv.weight      

19361

In [18]:
x = torch.zeros((1, 1, 256, 256, 256), dtype=torch.float32, device='cuda')
n_activs = project.model.count_activations(model, x)
n_activs

input                                    16777216	(1, 1, 256, 256, 256)	torch.float32
encoder.level0.conv_block.unit0.conv     134217728	(1, 8, 256, 256, 256)	torch.float32
encoder.level0.conv_block.unit0.norm     134217728	(1, 8, 256, 256, 256)	torch.float32
encoder.level0.conv_block.unit1.conv     134217728	(1, 8, 256, 256, 256)	torch.float32
encoder.level1.conv_block.unit0.conv     33554432	(1, 16, 128, 128, 128)	torch.float32
encoder.level1.conv_block.unit1.conv     33554432	(1, 16, 128, 128, 128)	torch.float32
decoder.level0.conv_block.unit1.conv     134217728	(1, 8, 256, 256, 256)	torch.float32


620756992

In [20]:
model_mib = (n_activs + n_params) * 2 * 4 / 2**20 # MiB
model_mib

4736.1477127075195

In [21]:
import torch.optim
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-3)
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.005
    maximize: False
    weight_decay: 0.01
)

In [23]:
import project.training
import project.evaluation
import warp as wp
wp.config.quiet = True
evaluator = project.evaluation.Evaluator()
trainer = project.training.Trainer(
    model, optimizer, train_loader, test_loader, val_loader, evaluator, solver_kws=dict(reg_weight=1e-4)
)

In [123]:
trainer.train(num_epochs=1)

Start training loop
Epoch 0/1
Start val phase


Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x14dee21d5940>
Traceback (most recent call last):
  File "/ocean/projects/asc170022p/mtragoza/mambaforge/envs/warp/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
<function _MultiProcessingDataLoaderIter.__del__ at 0x14dee21d5940>    self._shutdown_workers()Exception ignored in: 

  File "/ocean/projects/asc170022p/mtragoza/mambaforge/envs/warp/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers
<function _MultiProcessingDataLoaderIter.__del__ at 0x14dee21d5940>    Traceback (most recent call last):
if w.is_alive():

 Traceback (most recent call last):
    File "/ocean/projects/asc170022p/mtragoza/mambaforge/envs/warp/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    ^    ^^  File "/ocean/projects/asc170022p/mtragoza/mambaforge/envs/warp/lib/python3.11/site-packages/torch/util

Val batch 1/108


KeyError: 'u'